# Compute deformations

This notebook performs a geodesic interpolation between each pair of consecutive meshes.

The meshes are unparameterized surfaces, on which the generalized elastic metrics are used.

The geodesic itself is saved as new meshes. The geodesic shows how a mesh smoothly deforms onto the next mesh of the time-series.

This notebook should be run on the server where a GPU is available.

## Setup

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()
print("Working directory: ", work_dir)

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
h2_surfacematch_dir = os.path.join(work_dir, "H2_SurfaceMatch")
sys.path.append(h2_surfacematch_dir)

print("Added directories to syspath:")
print(sys_dir)
print(h2_surfacematch_dir)

Working directory:  /home/adele/code/my28brains
Added directories to syspath:
/home/adele/code
/home/adele/code/my28brains/H2_SurfaceMatch


## Imports

In [2]:
import glob
import time

import numpy as np
import torch
import trimesh

import my28brains.my28brains.io_mesh as io_mesh

INFO: Using numpy backend


In [3]:
import H2_SurfaceMatch.H2_match
import H2_SurfaceMatch.utils.input_output
import H2_SurfaceMatch.utils.utils

use_cuda = 1
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
torchdtype = torch.float32

print(torchdeviceId)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
cuda:0


## Define Dirs

In [4]:
CENTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "centered_meshes")
print("CENTERED_MESHES_DIR: ", CENTERED_MESHES_DIR)

CENTERED_NONDEGENERATE_MESHES_DIR = os.path.join(
    os.getcwd(), "data", "centered_nondegenerate_meshes"
)
print("CENTERED_NONDEGENERATE_MESHES_DIR: ", CENTERED_NONDEGENERATE_MESHES_DIR)
if not os.path.exists(CENTERED_NONDEGENERATE_MESHES_DIR):
    os.makedirs(CENTERED_NONDEGENERATE_MESHES_DIR)

CENTERED_MESHES_DIR:  /home/adele/code/my28brains/data/centered_meshes
CENTERED_NONDEGENERATE_MESHES_DIR:  /home/adele/code/my28brains/data/centered_nondegenerate_meshes


## Remove degenerate faces in all meshes

In [10]:
hemispheres = ["left", "right"]

structure_ids = list(range(1,10))
structure_ids.append(-1)

for hemisphere in hemispheres:
    for structure_id in structure_ids:

        string_base = os.path.join(
            # CENTERED_MESHES_DIR, f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply"
            CENTERED_MESHES_DIR, f"{hemisphere}_structure_{structure_id}**.ply"
        )
        paths = sorted(glob.glob(string_base))

        print(
            f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:"
        )
        for path in paths:
            print(path)
        
        for path in paths:
            print(f"\tLoad mesh from path: {path}")
            mesh = trimesh.load(path)
            new_vertices, new_faces = io_mesh.remove_degenerate_faces(mesh.vertices, mesh.faces)
            new_mesh = trimesh.Trimesh(vertices=new_vertices, faces=new_faces)

            ply_path = os.path.join(CENTERED_NONDEGENERATE_MESHES_DIR, os.path.basename(path))
            print(ply_path)
            io_mesh.write_trimesh_to_ply(new_mesh, ply_path)

Found 60 ply files for left hemisphere and anatomical structure 1:
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day01.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day02.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day03.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day04.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day05.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day06.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day07.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day08.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day09.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day10.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day11.ply
/home/adele/code/my28brains/data/centered_meshes/left_structure_1_day12.ply
/home/adele/code/my28

We will use the centered meshes, since they seemed more aligned than the registered meshes.

In [11]:
# for path in paths:
#     print(f"\tLoad mesh from path: {path}")
#     mesh = trimesh.load(path)
#     new_vertices, new_faces = io_mesh.remove_degenerate_faces(mesh.vertices, mesh.faces)
#     new_mesh = trimesh.Trimesh(vertices=new_vertices, faces=new_faces)

#     ply_path = os.path.join(CENTERED_NONDEGENERATE_MESHES_DIR, os.path.basename(path))
#     print(ply_path)
#     io_mesh.write_trimesh_to_ply(new_mesh, ply_path)

# Create sequence of ply showing smooth deformation by geodesic interpolation

In [12]:
#temporarily here, out of order
def _geodesic_interp(i_pair):
    """Auxiliary function that will be run in parallelon different GPUs."""
    start_time = time.time()
    start_path = start_paths[i_pair]
    end_path = end_paths[i_pair]

    [VS, FS, FunS] = H2_SurfaceMatch.utils.input_output.loadData(start_path)
    VS = VS / 10
    [VS, FS] = H2_SurfaceMatch.utils.input_output.decimate_mesh(
        VS, FS, int(FS.shape[0] / 4)
    )
    sources = [[VS, FS]]

    [VT, FT, FunT] = H2_SurfaceMatch.utils.input_output.loadData(end_path)
    VT = VT / 10
    [VT, FT] = H2_SurfaceMatch.utils.input_output.decimate_mesh(
        VT, FT, int(FT.shape[0] / 4)
    )
    targets = [[VT, FT]]

    source = sources[0]
    target = targets[0]

    geod, F0 = H2_SurfaceMatch.H2_match.H2MultiRes(
        source, target, a0, a1, b1, c1, d1, a2, resolutions=2, paramlist=paramlist
    )
    comp_time = time.time() - start_time
    print(f"Geodesic interpolation {i_pair} took: {comp_time / 60:.2f} minutes.")

    # We use the start mesh as the basename for the ply files
    ply_prefix = os.path.join(GEODESICS_DIR, os.path.basename(start_path))

    for i_time in range(geod.shape[0]):
        H2_SurfaceMatch.utils.input_output.plotGeodesic(
            [geod[i_time]],
            F0,
            stepsize=5,
            file_name=ply_prefix + "{}".format(i_time),
            axis=[0, 1, 0],
            angle=-1 * np.pi / 2,
        )
    print(f"Geodesic interpolation {i_pair} saved to: {GEODESICS_DIR}.")

In [13]:
# hemisphere = "left"
# structure_id = -1

hemispheres = ["left", "right"]

structure_ids = list(range(1,10))
structure_ids.append(-1)

for hemisphere in hemispheres:
    for structure_id in structure_ids:
        string_base = os.path.join(
            CENTERED_NONDEGENERATE_MESHES_DIR,
            f"{hemisphere}_structure_{structure_id}**.ply",
        )
        paths = sorted(glob.glob(string_base))

        print(
            f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:"
        )
        for path in paths:
            print(path)
        
        start_paths = paths[:-1]
        end_paths = paths[1:]
        
        a0 = 0.01
        a1 = 100
        b1 = 100
        c1 = 0.2
        d1 = 0.01
        a2 = 0.01


        param1 = {
            "weight_coef_dist_T": 10**1,
            "weight_coef_dist_S": 10**1,
            "sig_geom": 0.4,
            "max_iter": 2000,
            "time_steps": 2,
            "tri_unsample": True,
            "index": 0,
        }

        param2 = {
            "weight_coef_dist_T": 10**2,
            "weight_coef_dist_S": 10**2,
            "sig_geom": 0.3,
            "max_iter": 1000,
            "time_steps": 2,
            "tri_unsample": False,
            "index": 1,
        }

        param3 = {
            "weight_coef_dist_T": 10**3,
            "weight_coef_dist_S": 10**3,
            "sig_geom": 0.2,
            "max_iter": 1000,
            "time_steps": 2,
            "tri_unsample": False,
            "index": 1,
        }

        param4 = {
            "weight_coef_dist_T": 10**4,
            "weight_coef_dist_S": 10**4,
            "sig_geom": 0.1,
            "max_iter": 1000,
            "time_steps": 3,
            "tri_unsample": True,
            "index": 1,
        }

        param5 = {
            "weight_coef_dist_T": 10**5,
            "weight_coef_dist_S": 10**5,
            "sig_geom": 0.1,
            "max_iter": 1000,
            "time_steps": 4,
            "tri_unsample": False,
            "index": 2,
        }

        param6 = {
            "weight_coef_dist_T": 10**6,
            "weight_coef_dist_S": 10**6,
            "sig_geom": 0.05,
            "max_iter": 1000,
            "time_steps": 5,
            "tri_unsample": False,
            "index": 2,
        }


        paramlist = [param1, param2, param3, param4, param5, param6]
        
        GEODESICS_DIR = os.path.join(os.getcwd(), "data", "geodesics")
        print("GEODESICS_DIR: ", GEODESICS_DIR)
        if not os.path.exists(GEODESICS_DIR):
            os.makedirs(GEODESICS_DIR)
            
        for i_pair in range(len(start_paths)):
            print(f"\n\n -------> Geodesic interpolation for pair: {i_pair}/{len(start_paths)}")
            _geodesic_interp(i_pair)

Found 60 ply files for left hemisphere and anatomical structure 1:
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day01.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day02.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day03.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day04.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day05.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day06.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day07.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day08.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day09.ply
/home/adele/code/my28brains/data/centered_nondegenerate_meshes/left_structure_1_day10.ply
/home/adele/code/my28brains/data/

 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  258   1255   1313      1     0     0   5.927D-01   2.069D+01

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

 ############ Iteration 0:
F0.shape (80, 3)
geod.shape: (2, 43, 3)
in  triupsample:
iteration 0 in range(N) to subdivice mesh
iteration 1 in range(N) to subdivice mesh
len(geod): 2
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          978     M =           10



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  978   1000   1088      1     0     0   4.682D+01   5.809D+01

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 

 ############ Iteration 1:
F0.shape (320, 3)
geod.shape: (2, 163, 3)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          978     M =           10


...


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 3504     50    130      2     0     0         NaN         NaN

ABNORMAL_TERMINATION_IN_LNSRCH                              
geod.shape: (4, 584, 3)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7008     M =           10



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
 This problem is unconstrained.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 7008      1     21      1     0     0         NaN         NaN

ABNORMAL_TERMINATION_IN_LNSRCH                              

 ############ Iteration 5:
F0.shape (1152, 3)
geod.shape: (4, 584, 3)
in timesteps:
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5256     M =           10

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized 

 This problem is unconstrained.


KeyboardInterrupt: 

In [ ]:
# start_paths = paths[:-1]
# end_paths = paths[1:]

Can we force the interp on one given GPU? it seems, in any case, that it is using all CPUs... thus might not help

In [ ]:
# a0 = 0.01
# a1 = 100
# b1 = 100
# c1 = 0.2
# d1 = 0.01
# a2 = 0.01


# param1 = {
#     "weight_coef_dist_T": 10**1,
#     "weight_coef_dist_S": 10**1,
#     "sig_geom": 0.4,
#     "max_iter": 2000,
#     "time_steps": 2,
#     "tri_unsample": True,
#     "index": 0,
# }

# param2 = {
#     "weight_coef_dist_T": 10**2,
#     "weight_coef_dist_S": 10**2,
#     "sig_geom": 0.3,
#     "max_iter": 1000,
#     "time_steps": 2,
#     "tri_unsample": False,
#     "index": 1,
# }

# param3 = {
#     "weight_coef_dist_T": 10**3,
#     "weight_coef_dist_S": 10**3,
#     "sig_geom": 0.2,
#     "max_iter": 1000,
#     "time_steps": 2,
#     "tri_unsample": False,
#     "index": 1,
# }

# param4 = {
#     "weight_coef_dist_T": 10**4,
#     "weight_coef_dist_S": 10**4,
#     "sig_geom": 0.1,
#     "max_iter": 1000,
#     "time_steps": 3,
#     "tri_unsample": True,
#     "index": 1,
# }

# param5 = {
#     "weight_coef_dist_T": 10**5,
#     "weight_coef_dist_S": 10**5,
#     "sig_geom": 0.1,
#     "max_iter": 1000,
#     "time_steps": 4,
#     "tri_unsample": False,
#     "index": 2,
# }

# param6 = {
#     "weight_coef_dist_T": 10**6,
#     "weight_coef_dist_S": 10**6,
#     "sig_geom": 0.05,
#     "max_iter": 1000,
#     "time_steps": 5,
#     "tri_unsample": False,
#     "index": 2,
# }


# paramlist = [param1, param2, param3, param4, param5, param6]

In [ ]:
# GEODESICS_DIR = os.path.join(os.getcwd(), "data", "geodesics")
# print("GEODESICS_DIR: ", GEODESICS_DIR)
# if not os.path.exists(GEODESICS_DIR):
#     os.makedirs(GEODESICS_DIR)

### Function performing one interpolation

In [13]:
# def _geodesic_interp(i_pair):
#     """Auxiliary function that will be run in parallelon different GPUs."""
#     start_time = time.time()
#     start_path = start_paths[i_pair]
#     end_path = end_paths[i_pair]

#     [VS, FS, FunS] = H2_SurfaceMatch.utils.input_output.loadData(start_path)
#     VS = VS / 10
#     [VS, FS] = H2_SurfaceMatch.utils.input_output.decimate_mesh(
#         VS, FS, int(FS.shape[0] / 4)
#     )
#     sources = [[VS, FS]]

#     [VT, FT, FunT] = H2_SurfaceMatch.utils.input_output.loadData(end_path)
#     VT = VT / 10
#     [VT, FT] = H2_SurfaceMatch.utils.input_output.decimate_mesh(
#         VT, FT, int(FT.shape[0] / 4)
#     )
#     targets = [[VT, FT]]

#     source = sources[0]
#     target = targets[0]

#     geod, F0 = H2_SurfaceMatch.H2_match.H2MultiRes(
#         source, target, a0, a1, b1, c1, d1, a2, resolutions=2, paramlist=paramlist
#     )
#     comp_time = time.time() - start_time
#     print(f"Geodesic interpolation {i_pair} took: {comp_time / 60:.2f} minutes.")

#     # We use the start mesh as the basename for the ply files
#     ply_prefix = os.path.join(GEODESICS_DIR, os.path.basename(start_path))

#     for i_time in range(geod.shape[0]):
#         H2_SurfaceMatch.utils.input_output.plotGeodesic(
#             [geod[i_time]],
#             F0,
#             stepsize=5,
#             file_name=ply_prefix + "{}".format(i_time),
#             axis=[0, 1, 0],
#             angle=-1 * np.pi / 2,
#         )
#     print(f"Geodesic interpolation {i_pair} saved to: {geod_dir}.")

This code ran until "Geodesic interpolation for pair: 12/26", then failed at iteration 5 with error:

```
File ~/code/H2_SurfaceMatch/enr/H2.py:135, in getGabNorm(alpha, xi, g, dg, dn, a, b, c, d)
    132 areas=torch.sqrt(torch.det(g)).to(dtype=torchdtype, device=torchdeviceId)
    133 # This inversion fails
    134 # g is getSurfMetric(midpoints[i],F_sol)
--> 135 ginv=torch.inverse(g)
    136 ginvdg=torch.matmul(ginv,dg)    
    137 A=0

_LinAlgError: linalg.inv: (Batch element 1434): The diagonal element 2 is zero, the inversion could not be completed because the input matrix is singular.
```

In [14]:
# for i_pair in range(len(start_paths)):
#     print(f"\n\n -------> Geodesic interpolation for pair: {i_pair}/{len(start_paths)}")
#     _geodesic_interp(i_pair)